### Brain extraction(Skull stripping)

**Learning outcomes:**
- How to do quick brain extraction using ants(antspynet module)

In [1]:
%matplotlib inline

import os
from help_functions import *

import ants
import SimpleITK as sitk

print(f'AntsPy version = {ants.__version__}')
print(f'SimpleITK version = {sitk.__version__}')

AntsPy version = 0.4.2
SimpleITK version = 2.3.1


In [2]:
BASE_DIR = os.path.dirname(os.path.dirname(os.path.abspath("__file__")))
print(f'project folder = {BASE_DIR}')

project folder = c:\Users\Windows\Desktop\GBM_Segmentation


#### Raw Image

In [3]:
raw_img_path = os.path.join(BASE_DIR, 'Brain_Extraction', 'Template', 'mni_icbm152_nlin_sym_09a', 'brain-lesion_T1w.nii')
raw_img_ants = ants.image_read(raw_img_path, reorient='IAL')

print(f'shape = {raw_img_ants.numpy().shape} -> (Z, X, Y)')

explore_3D_array(arr=raw_img_ants.numpy(), cmap='nipy_spectral')

shape = (192, 192, 160) -> (Z, X, Y)


interactive(children=(IntSlider(value=95, description='SLICE', max=191), Output()), _dom_classes=('widget-inte…

#### Deep Learning based method

#### Load Model via AntsPyNet API and predict

In [4]:
from antspynet.utilities import brain_extraction

In [7]:
prob_brain_mask = brain_extraction(raw_img_ants, modality="t1", verbose=True,)

Brain extraction:  retrieving model weights.
5683832/5683832 [==============================] - 2s 0us/step
Brain extraction:  retrieving template.
14969865/14969865 [==============================] - 3s 0us/step

Brain extraction:  normalizing image to the template.
Brain extraction:  prediction and decoding.
1/1 [==============================] - 6s 6s/step
Brain extraction:  renormalize probability mask to native space.


#### Inspect probabilities array

In [8]:
print(prob_brain_mask)
explore_3D_array(prob_brain_mask.numpy())

ANTsImage (IAL)
	 Pixel Type : float (float32)
	 Components : 1
	 Dimensions : (192, 192, 160)
	 Spacing    : (1.25, 1.25, 1.2)
	 Origin     : (98.1114, -149.1525, -129.5975)
	 Direction  : [-0.  0. -1.  0.  1.  0.  1.  0.  0.]



interactive(children=(IntSlider(value=95, description='SLICE', max=191), Output()), _dom_classes=('widget-inte…

#### Generate final mask

In [9]:
brain_mask = ants.get_mask(prob_brain_mask, low_thresh=0.5)

In [10]:
explore_3D_array_with_mask_contour(raw_img_ants.numpy(), brain_mask.numpy())

interactive(children=(IntSlider(value=95, description='SLICE', max=191), Output()), _dom_classes=('widget-inte…

#### Generate brain masked

In [11]:
masked = ants.mask_image(raw_img_ants, brain_mask)

explore_3D_array(masked.numpy())

interactive(children=(IntSlider(value=95, description='SLICE', max=191), Output()), _dom_classes=('widget-inte…